### **Organize PSD Data**

- [ ]  Ensure all power spectral data (PSD) is saved per **subject-session** in a consistent format:
    - One file or object per subject-session
    - Contains:
        - EEG **channels**
        - **Frequencies**
        - Power per **epoch** or pre-averaged by condition
    - Associated **metadata**:
        - Subject ID
        - Session ID 
        - Dataset name
        - Cognitive **state label** (OT, MW)

In [1]:
from eeg_analyzer.dataset import Dataset
from utils.config import DATASETS

dataset_config = DATASETS['touryan2022']

dataset = Dataset(dataset_config)
dataset.load_subjects()

### **Extract Alpha Power**

- [ ]  For each subject-session
    - Select **8–12 Hz** frequency band
    - **Sum power across frequencies** per channel and epoch
- [ ]  Result per epoch:

```python
{
    "subject_session": "001_1",
    "subject_id": "001",
    "session_id": "1",
    "channel": "Pz",
    "task": "SART",
    "state": "MW",
    "alpha_power": 3.45,
    ...
}
```

### **Assign Numerical Condition Labels**

For modeling the *ordinal direction* of alpha power:

| State | Code |
| --- | --- |
| OT | 0 |
| MW | 1 |
| MED | 2 |
- [ ]  For **Jin** and **Touryan**, you only use codes `0` and `1` (OT, MW)
- [ ]  For **Braboszcz**, use codes `1` and `2` (MW, MED)

In [2]:
# Get all list of all epochs for all subject-session pairs as dict with these keys:
# 'subject_session', 'subject', 'session', 'channel', 'task', 'state', 'band_power'

epochs = dataset.to_long_band_power_list(freq_band=(8,12))  # Alpha band

estimated_length = dataset.estimate_long_band_power_length()

### **Create Long-Form DataFrame per Dataset**

For each dataset, prepare:
| subject_session | subject_id | group | channel/ROI | state | alpha_power |
| --- | --- | --- | --- | --- | --- |
| 001_1 | 001 | NaN | Pz | 0 | 3.24 |
| 001_1 | 001 | NaN | Pz | 1 | 3.88 |
| 060_1 | 060 | vip | Pz | 1 | 5.12 |
| 060_1 | 060 | vip | Pz | 2 | 6.00 |

In [6]:
# create a dataframe with the epochs
import pandas as pd
import numpy as np
df_full = pd.DataFrame(epochs)

def exlude_subjects(df, subjects):
    """
    Exclude subjects from the dataframe.
    """
    for subject in subjects:
        df = df[df['subject_id'] != subject]
    return df

# Exclude all subjects with a state ratio r<1/9 and epochs < median number of epochs
subject_to_exclude = []
total_median_epochs = np.median([dataset.get_total_epochs(subject_id) for subject_id in dataset.subject_IDs])
print(f"Total median epochs: {total_median_epochs}")
for subject_id in dataset.subject_IDs:
    state_ratio = dataset.get_state_ratio(subject_id)
    total_epochs = dataset.get_total_epochs(subject_id)
    if state_ratio < 1/9 and total_epochs < total_median_epochs:
        print(f"Subject {subject_id}: {state_ratio} with total epochs: {total_epochs}. Subject excluded.")
        subject_to_exclude.append(subject_id)

df_full = exlude_subjects(df_full, subject_to_exclude)
        
        
df_full.head()

Total median epochs: 22.0
Subject 05: 0.06666666666666667 with total epochs: 16. Subject excluded.
Subject 06: 0.058823529411764705 with total epochs: 18. Subject excluded.
Subject 08: 0.08333333333333333 with total epochs: 13. Subject excluded.
Subject 09: 0.0 with total epochs: 18. Subject excluded.
Subject 18: 0.0 with total epochs: 19. Subject excluded.
Subject 20: 0.10526315789473684 with total epochs: 21. Subject excluded.


,subject_session,subject_id,session_id,group,epoch_idx,channel,cortical_region,hemisphere,task,state,band_power,is_bad
0,01_2,01,2,None,0,Fp1,prefrontal,left,police_detection,0,2.116215,False
1,01_2,01,2,None,0,AF7,prefrontal,left,police_detection,0,1.410603,False
2,01_2,01,2,None,0,AF3,prefrontal,left,police_detection,0,2.347347,False
3,01_2,01,2,None,0,F1,frontal,left,police_detection,0,3.197372,False
4,01_2,01,2,None,0,F3,frontal,left,police_detection,0,3.248961,False


## **Report the state balance**

In [7]:
state_counts = df_full['state'].value_counts()
state_counts = state_counts / state_counts.sum()
print(f"State balance: {state_counts.to_dict()}")

State balance: {0: 0.8734939759036144, 1: 0.12650602409638553}


## **Make grouped DataFrames per channel**

* [ ] Create a list of DataFrames for each its own channel.
* [ ] Create a new column with the z-score within state.
* [ ] Filter the data by removing all epochs with a z-score > 4.

In [8]:
# Create a list of dataframes for each channel (keep the channel column)
df_channels = []
for channel in df_full['channel'].unique():
    df_channel = df_full[df_full['channel'] == channel].reset_index(drop=True)
    df_channels.append(df_channel)

# create a new column with z-scores within each state
# def z_score_within_state(df):
#     grouped = df.groupby(['subject_session', 'state'])
#     df['z_score'] = grouped['band_power'].transform(lambda x: (x - x.mean()) / x.std())
#     return df

# # apply the z-score function to each channel dataframe
# df_channels_z = []
# for df_channel in df_channels:
#     df_channel_z = z_score_within_state(df_channel)
#     df_channels_z.append(df_channel_z)

# # Remove all rows where z_score > 3
# for i in range(len(df_channels_z)):
#     df_channels_z[i] = df_channels_z[i][df_channels_z[i]['z_score'] <= 3]

# print(df_channels_z[0].head())


### **Fit Mixed Effects Model per Channel**

Do this **separately for each dataset** and for each channel (or ROI):

### ✅ Model formula:

```python
alpha_power ~ state + (state | subject_session)
```

This:

- Estimates the effect of `state` (e.g., OT → MW or MW → MED)
- Models per-subject variability in both baseline (intercept) and sensitivity (slope)

In [9]:
# Fit mixed effects model per channel
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

results_per_channel = {}

for df_channel in df_channels:
    channel_name = df_channel['channel'].iloc[0] if 'channel' in df_channel else None
    # Fit the mixed effects model: alpha_power ~ state + (state | subject_session)
    # Use 'band_power' as the dependent variable
    # 'state' as fixed effect, random intercept and slope for 'subject_session'
    model = smf.mixedlm(
        "band_power ~ state",
        df_channel,
        groups="subject_session",
        re_formula="~state"
    )
    result = model.fit(method="lbfgs")
    results_per_channel[channel_name] = result

# Print summary for all channels
for channel in results_per_channel:
    print(f"Results for channel: {channel}")
    print(results_per_channel[channel].summary())

/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, 

Results for channel: Fp1
                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     band_power
No. Observations:       332         Method:                 REML      
No. Groups:             14          Scale:                  47.8955   
Min. group size:        19          Log-Likelihood:         -1121.6755
Max. group size:        28          Converged:              Yes       
Mean group size:        23.7                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                    5.689    0.894  6.361 0.000  3.936  7.442
state                       -0.672    1.171 -0.574 0.566 -2.966  1.622
subject_session Var          8.834    0.622                           
subject_session x state Cov -5.081    0.770                          

/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2704: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2704: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))


                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     band_power
No. Observations:       332         Method:                 REML      
No. Groups:             14          Scale:                  20.4882   
Min. group size:        19          Log-Likelihood:         -992.2776 
Max. group size:        28          Converged:              Yes       
Mean group size:        23.7                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                    6.591    1.203  5.476 0.000  4.232  8.949
state                       -0.120    0.736 -0.162 0.871 -1.563  1.324
subject_session Var         19.265    2.207                           
subject_session x state Cov  4.201    2.237                           
state Var              

# Z-score normalize whitin-subject
Z-score normalize whitin-subject across epochs and fit new models for easier interpretations 

In [7]:
# create a new column with z-scores within each state
def z_score_within_state(df):
    grouped = df.groupby(['subject_session'])
    df['z_score'] = grouped['band_power'].transform(lambda x: (x - x.mean()) / x.std())
    return df

# apply the z-score function to each channel dataframe
df_channels_z = []
for df_channel in df_channels:
    df_channel_z = z_score_within_state(df_channel)
    df_channels_z.append(df_channel_z)

In [8]:
# Fit mixed effects model per channel with z-score insted of band_power

results_per_channel_z = {}

for df_channel in df_channels_z:
    channel_name = df_channel['channel'].iloc[0] if 'channel' in df_channel else None
    # Fit the mixed effects model: alpha_power ~ state + (state | subject_session)
    # Use 'band_power' as the dependent variable
    # 'state' as fixed effect, random intercept and slope for 'subject_session'
    model = smf.mixedlm(
        "z_score ~ state",
        df_channel,
        groups="subject_session",
        re_formula="~state"
    )
    result = model.fit(method="lbfgs")
    results_per_channel_z[channel_name] = result

# Print summary for all channels
for channel in results_per_channel_z:
    print(f"Results for channel: {channel}")
    print(results_per_channel_z[channel].summary())

/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, 

Results for channel: Fp1
                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     z_score    
No. Observations:      11103       Method:                 REML       
No. Groups:            50          Scale:                  0.9181     
Min. group size:       150         Log-Likelihood:         -15359.0241
Max. group size:       270         Converged:              Yes        
Mean group size:       222.1                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                   -0.043    0.041 -1.050 0.294 -0.125  0.038
state                        0.091    0.081  1.131 0.258 -0.067  0.249
subject_session Var          0.077    0.022                           
subject_session x state Cov -0.155    0.038                          

In [9]:
# Fit mixed effects model per channel with z-score insted of band_power

results_per_channel_z = {}

for df_channel in df_channels_z:
    channel_name = df_channel['channel'].iloc[0] if 'channel' in df_channel else None
    # Fit the mixed effects model: alpha_power ~ state + (state | subject_session)
    # Use 'band_power' as the dependent variable
    # 'state' as fixed effect, random intercept and slope for 'subject_session'
    model = smf.mixedlm(
        "z_score ~ state",
        df_channel,
        groups="subject_session",
        re_formula="~1"
    )
    result = model.fit(method="lbfgs")
    results_per_channel_z[channel_name] = result

# Print summary for all channels
for channel in results_per_channel_z:
    print(f"Results for channel: {channel}")
    print(results_per_channel_z[channel].summary())

/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/home/sivert/anaconda3/envs/mne-env/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the e

Results for channel: Fp1
                  Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       z_score
No. Observations:         11103         Method:                   REML   
No. Groups:               50            Scale:                    0.9936 
Min. group size:          150           Log-Likelihood:           inf    
Max. group size:          270           Converged:                Yes    
Mean group size:          222.1                                          
-------------------------------------------------------------------------
                    Coef.   Std.Err.    z    P>|z|    [0.025     0.975]  
-------------------------------------------------------------------------
Intercept           -0.001 208459.628 -0.000 1.000 -408573.364 408573.361
state                0.093      0.019  4.866 0.000       0.055      0.130
subject_session Var  0.000                                               

Results for channel: AF7
     

### **Store & Summarize Results**

For each model/channel:

- [ ]  Store:
    - Fixed effect estimate for `state`
    - p-value
    - t-statistic (optional)
    - Number of subjects included
- [ ]  Save results to `.csv` or `.json`

### **Visualize Results**

- [ ]  Plot per-channel **bar plots** or **line plots** of alpha power by condition
- [ ]  Plot **topoplots** (scalp maps) of:
    - `state` slope per channel
    - p-values (FDR-corrected or raw, with masking)
    - t-values

In [ ]:
import matplotlib.pyplot as plt
from utils.config import PLOTS_PATH
import os



### **Correct for Multiple Comparisons**

- [ ]  Across channels (e.g., for scalp maps), correct p-values using:
    - **False Discovery Rate (FDR)**
    - Or **cluster-based permutation test** if possible

### **Filter Low-Effect Subjects**

- [ ]  Compute within-subject effect sizes (e.g., Cohen’s *d*) between states
- [ ]  Re-run the model excluding sessions with **d < 0.5**
- [ ]  Compare results to check robustness